# <center>MobileNet - Pytorch

# Step 1: Prepare data

In [1]:
# MobileNet-Pytorch
import argparse 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
#from mobilenets import mobilenet

use_cuda = torch.cuda.is_available()
use_cudause_cud  = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [2]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

valid_size=0.1

# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=train_transform)

valid_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=valid_transform)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train)) #5w张图片的10%用来当做验证集


np.random.seed(20)# 42
np.random.shuffle(indices) # 随机乱序[0,1,...,49999]

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx) # 这个很有意思
valid_sampler = SubsetRandomSampler(valid_idx)

###################################################################################
# ------------------------- 使用不同的批次大小 ------------------------------------
###################################################################################

show_step=10  # 批次大，show_step就小点
max_epoch=60  # 训练最大epoch数目

train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=256, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                batch_size=256, sampler=valid_sampler)


test_transform = transforms.Compose([
    transforms.ToTensor(), normalize
])

test_dataset = datasets.CIFAR10(root="data", 
                                train=False, 
                                download=True,transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=64, 
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Step 2: Model Config

# 32  缩放5次到 1x1@1024 
# From https://github.com/kuangliu/pytorch-cifar 
import torch
import torch.nn as nn
import torch.nn.functional as F


class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        
        #self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        one_conv_kernel_size = 3
        self.conv1D= nn.Conv1d(1, out_planes, one_conv_kernel_size, stride=1,padding=1,groups=1,dilation=1,bias=False) # 在__init__初始化        
        
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        
        out = F.relu(self.bn1(self.conv1(x)))
        
        # -------------------------- Attention -----------------------
        w = F.avg_pool2d(x,x.shape[-1])  #最好在初始化层定义好
        #print(w.shape)
        # [bs,in_Channel,1,1]
        w = w.view(w.shape[0],1,w.shape[1])
        # [bs,1,in_Channel]
        # one_conv_filter = nn.Conv1d(1, out_channel, one_conv_kernel_size, stride=1,padding=1,groups=1,dilation=1) # 在__init__初始化
        # [bs,out_channel,in_Channel]
        w = self.conv1D(w)
        w = 0.5*F.tanh(w) # [-0.5,+0.5]
        # -------------- softmax ---------------------------
        #print(w.shape)
        w = w.view(w.shape[0],w.shape[1],w.shape[2],1,1)
        #print(w.shape)
        
        # ------------------------- fusion --------------------------
        out=out.view(out.shape[0],1,out.shape[1],out.shape[2],out.shape[3])
        #print("x size:",out.shape)
        
        out=out*w
        #print("after fusion x size:",out.shape)
        out=out.sum(dim=2)
        
        out = F.relu(self.bn2(out))
        
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]

    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32) # 自动化构建层
        self.linear = nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [3]:
# 32  缩放5次到 1x1@1024 
# From https://github.com/kuangliu/pytorch-cifar 
import torch
import torch.nn as nn
import torch.nn.functional as F

class Block_Attention(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block_Attention, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        
        #self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        one_conv_kernel_size = 3
        self.conv1D= nn.Conv1d(1, out_planes, one_conv_kernel_size, stride=1,padding=1,groups=1,dilation=1,bias=False) # 在__init__初始化        
        
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        
        out = F.relu(self.bn1(self.conv1(x)))
        
        # -------------------------- Attention -----------------------
        w = F.avg_pool2d(x,x.shape[-1])  #最好在初始化层定义好
        #print(w.shape)
        # [bs,in_Channel,1,1]
        w = w.view(w.shape[0],1,w.shape[1])
        # [bs,1,in_Channel]
        # one_conv_filter = nn.Conv1d(1, out_channel, one_conv_kernel_size, stride=1,padding=1,groups=1,dilation=1) # 在__init__初始化
        # [bs,out_channel,in_Channel]
        w = self.conv1D(w)
        w = 0.5*F.tanh(w) # [-0.5,+0.5]
        # -------------- softmax ---------------------------
        #print(w.shape)
        w = w.view(w.shape[0],w.shape[1],w.shape[2],1,1)
        #print(w.shape)
        
        # ------------------------- fusion --------------------------
        out=out.view(out.shape[0],1,out.shape[1],out.shape[2],out.shape[3])
        #print("x size:",out.shape)
        
        out=out*w
        #print("after fusion x size:",out.shape)
        out=out.sum(dim=2)
        
        out = F.relu(self.bn2(out))
        
        return out


class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    #cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]
    cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), [1024,1]]
    
    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32) # 自动化构建层
        self.linear = nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            if isinstance(x, int):
                out_planes = x
                stride = 1 
                layers.append(Block(in_planes, out_planes, stride))
            elif isinstance(x, tuple):
                out_planes = x[0]
                stride = x[1]
                layers.append(Block(in_planes, out_planes, stride))
            # AC层通过list存放设置参数
            elif isinstance(x, list):
                out_planes= x[0]
                stride = x[1] if len(x)==2 else 1
                layers.append(Block_Attention(in_planes, out_planes, stride))   
            else:
                pass
            
            in_planes = out_planes
            
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [4]:
# From https://github.com/Z0m6ie/CIFAR-10_PyTorch
#model = mobilenet(num_classes=10, large_img=False)

# From https://github.com/kuangliu/pytorch-cifar 
if torch.cuda.is_available():
    model=MobileNet(10).cuda()
else:
    model=MobileNet(10)

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

In [5]:
# Implement validation
def train(epoch):
    model.train()
    #writer = SummaryWriter()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        correct = 0
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()
        
        loss = criterion(output, target)
        loss.backward()
        accuracy = 100. * (correct.cpu().numpy()/ len(output))
        optimizer.step()
        if batch_idx % show_step == 0:
#             if batch_idx % 2*show_step == 0:
#                 print(model.layers[1].conv1D.weight.shape)
#                 print(model.layers[1].conv1D.weight[0:2][0:2])
            
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), accuracy))
            #writer.add_scalar('Loss/Loss', loss.item(), epoch)
            #writer.add_scalar('Accuracy/Accuracy', accuracy, epoch)
    scheduler.step()

In [6]:
def validate(epoch):
    model.eval()
    #writer = SummaryWriter()
    valid_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        valid_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    valid_loss /= len(valid_idx)
    accuracy = 100. * correct.cpu().numpy() / len(valid_idx)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        valid_loss, correct, len(valid_idx),
        100. * correct / len(valid_idx)))
    #writer.add_scalar('Loss/Validation_Loss', valid_loss, epoch)
    #writer.add_scalar('Accuracy/Validation_Accuracy', accuracy, epoch)
    return valid_loss, accuracy

In [7]:
# Fix best model

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct.cpu().numpy() / len(test_loader.dataset)))

In [ ]:
def save_best(loss, accuracy, best_loss, best_acc):
    if best_loss == None:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
        
    elif loss < best_loss and accuracy > best_acc:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    return best_loss, best_acc

In [ ]:
# Fantastic logger for tensorboard and pytorch, 
# run tensorboard by opening a new terminal and run "tensorboard --logdir runs"
# open tensorboard at http://localhost:6006/
from tensorboardX import SummaryWriter
best_loss = None
best_acc = None

import time 
SINCE=time.time()

for epoch in range(max_epoch):
    train(epoch)
    
    loss, accuracy = validate(epoch)
    best_loss, best_acc = save_best(loss, accuracy, best_loss, best_acc)
    
    NOW=time.time() 
    DURINGS=NOW-SINCE
    SINCE=NOW
    print("the time of this epoch:[{} s]".format(DURINGS))
    
# writer = SummaryWriter()
# writer.export_scalars_to_json("./all_scalars.json")

# writer.close()

#---------------------------- Test ------------------------------
test(epoch)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.306598, Accuracy: 6.64
Train Epoch: 0 [2560/50000 (6%)]	Loss: 2.238290, Accuracy: 15.62
Train Epoch: 0 [5120/50000 (11%)]	Loss: 2.286614, Accuracy: 16.41
Train Epoch: 0 [7680/50000 (17%)]	Loss: 2.175904, Accuracy: 18.36
Train Epoch: 0 [10240/50000 (23%)]	Loss: 2.049612, Accuracy: 24.61
Train Epoch: 0 [12800/50000 (28%)]	Loss: 1.972886, Accuracy: 22.66
Train Epoch: 0 [15360/50000 (34%)]	Loss: 1.906119, Accuracy: 32.42
Train Epoch: 0 [17920/50000 (40%)]	Loss: 1.893017, Accuracy: 24.22
Train Epoch: 0 [20480/50000 (45%)]	Loss: 1.913116, Accuracy: 25.39
Train Epoch: 0 [23040/50000 (51%)]	Loss: 1.847740, Accuracy: 27.73
Train Epoch: 0 [25600/50000 (57%)]	Loss: 1.793561, Accuracy: 28.52
Train Epoch: 0 [28160/50000 (62%)]	Loss: 1.864463, Accuracy: 24.22
Train Epoch: 0 [30720/50000 (68%)]	Loss: 1.682755, Accuracy: 31.64
Train Epoch: 0 [33280/50000 (74%)]	Loss: 1.740243, Accuracy: 30.86
Train Epoch: 0 [35840/50000 (80%)]	Loss: 1.687431, Accuracy: 30.47
Train

Train Epoch: 6 [12800/50000 (28%)]	Loss: 0.843795, Accuracy: 73.83
Train Epoch: 6 [15360/50000 (34%)]	Loss: 0.766178, Accuracy: 75.78
Train Epoch: 6 [17920/50000 (40%)]	Loss: 0.709176, Accuracy: 74.22
Train Epoch: 6 [20480/50000 (45%)]	Loss: 0.793737, Accuracy: 73.05
Train Epoch: 6 [23040/50000 (51%)]	Loss: 0.755095, Accuracy: 75.78
Train Epoch: 6 [25600/50000 (57%)]	Loss: 0.700138, Accuracy: 77.34
Train Epoch: 6 [28160/50000 (62%)]	Loss: 0.620458, Accuracy: 77.73
Train Epoch: 6 [30720/50000 (68%)]	Loss: 0.649903, Accuracy: 75.78
Train Epoch: 6 [33280/50000 (74%)]	Loss: 0.761375, Accuracy: 74.22
Train Epoch: 6 [35840/50000 (80%)]	Loss: 0.619505, Accuracy: 78.91
Train Epoch: 6 [38400/50000 (85%)]	Loss: 0.631774, Accuracy: 79.30
Train Epoch: 6 [40960/50000 (91%)]	Loss: 0.762369, Accuracy: 74.22
Train Epoch: 6 [43520/50000 (97%)]	Loss: 0.729069, Accuracy: 73.83

Validation set: Average loss: 1.0533, Accuracy: 3455/5000 (69.00%)

the time of this epoch:[158.28313779830933 s]
Train Epoch: 7

Train Epoch: 12 [25600/50000 (57%)]	Loss: 0.490528, Accuracy: 83.59
Train Epoch: 12 [28160/50000 (62%)]	Loss: 0.405391, Accuracy: 86.33
Train Epoch: 12 [30720/50000 (68%)]	Loss: 0.355667, Accuracy: 87.11
Train Epoch: 12 [33280/50000 (74%)]	Loss: 0.509631, Accuracy: 83.59
Train Epoch: 12 [35840/50000 (80%)]	Loss: 0.435676, Accuracy: 85.55
Train Epoch: 12 [38400/50000 (85%)]	Loss: 0.475741, Accuracy: 84.38
Train Epoch: 12 [40960/50000 (91%)]	Loss: 0.426568, Accuracy: 87.11
Train Epoch: 12 [43520/50000 (97%)]	Loss: 0.487487, Accuracy: 82.81

Validation set: Average loss: 0.5310, Accuracy: 4090/5000 (81.00%)

the time of this epoch:[158.39070653915405 s]
Train Epoch: 13 [0/50000 (0%)]	Loss: 0.478984, Accuracy: 83.20
Train Epoch: 13 [2560/50000 (6%)]	Loss: 0.459093, Accuracy: 85.16
Train Epoch: 13 [5120/50000 (11%)]	Loss: 0.423320, Accuracy: 84.38
Train Epoch: 13 [7680/50000 (17%)]	Loss: 0.432855, Accuracy: 86.33
Train Epoch: 13 [10240/50000 (23%)]	Loss: 0.404462, Accuracy: 82.81
Train Epoc

Train Epoch: 18 [35840/50000 (80%)]	Loss: 0.352128, Accuracy: 89.45
Train Epoch: 18 [38400/50000 (85%)]	Loss: 0.343875, Accuracy: 89.06
Train Epoch: 18 [40960/50000 (91%)]	Loss: 0.285013, Accuracy: 88.67
Train Epoch: 18 [43520/50000 (97%)]	Loss: 0.377868, Accuracy: 86.33

Validation set: Average loss: 0.4500, Accuracy: 4257/5000 (85.00%)

the time of this epoch:[158.41813039779663 s]
Train Epoch: 19 [0/50000 (0%)]	Loss: 0.315110, Accuracy: 89.06
Train Epoch: 19 [2560/50000 (6%)]	Loss: 0.315498, Accuracy: 86.72
Train Epoch: 19 [5120/50000 (11%)]	Loss: 0.269358, Accuracy: 91.41
Train Epoch: 19 [7680/50000 (17%)]	Loss: 0.366230, Accuracy: 89.45
Train Epoch: 19 [10240/50000 (23%)]	Loss: 0.331873, Accuracy: 90.62
Train Epoch: 19 [12800/50000 (28%)]	Loss: 0.432238, Accuracy: 88.28
Train Epoch: 19 [15360/50000 (34%)]	Loss: 0.423635, Accuracy: 86.72
Train Epoch: 19 [17920/50000 (40%)]	Loss: 0.362628, Accuracy: 87.11
Train Epoch: 19 [20480/50000 (45%)]	Loss: 0.271403, Accuracy: 90.23
Train Epoc


Validation set: Average loss: 0.4038, Accuracy: 4351/5000 (87.00%)

the time of this epoch:[158.42547941207886 s]
Train Epoch: 25 [0/50000 (0%)]	Loss: 0.193637, Accuracy: 92.97
Train Epoch: 25 [2560/50000 (6%)]	Loss: 0.208625, Accuracy: 92.19
Train Epoch: 25 [5120/50000 (11%)]	Loss: 0.204928, Accuracy: 92.58
Train Epoch: 25 [7680/50000 (17%)]	Loss: 0.224158, Accuracy: 92.97
Train Epoch: 25 [10240/50000 (23%)]	Loss: 0.201792, Accuracy: 93.75
Train Epoch: 25 [12800/50000 (28%)]	Loss: 0.204006, Accuracy: 91.80
Train Epoch: 25 [15360/50000 (34%)]	Loss: 0.288245, Accuracy: 89.06
Train Epoch: 25 [17920/50000 (40%)]	Loss: 0.280435, Accuracy: 90.23
Train Epoch: 25 [20480/50000 (45%)]	Loss: 0.182546, Accuracy: 92.19
Train Epoch: 25 [23040/50000 (51%)]	Loss: 0.178077, Accuracy: 92.58
Train Epoch: 25 [25600/50000 (57%)]	Loss: 0.273640, Accuracy: 90.23
Train Epoch: 25 [28160/50000 (62%)]	Loss: 0.250088, Accuracy: 88.28
Train Epoch: 25 [30720/50000 (68%)]	Loss: 0.227684, Accuracy: 92.97
Train Epoc

Train Epoch: 31 [5120/50000 (11%)]	Loss: 0.174064, Accuracy: 94.14
Train Epoch: 31 [7680/50000 (17%)]	Loss: 0.158650, Accuracy: 94.14
Train Epoch: 31 [10240/50000 (23%)]	Loss: 0.169074, Accuracy: 93.75
Train Epoch: 31 [12800/50000 (28%)]	Loss: 0.161835, Accuracy: 94.92
Train Epoch: 31 [15360/50000 (34%)]	Loss: 0.175946, Accuracy: 93.36
Train Epoch: 31 [17920/50000 (40%)]	Loss: 0.209860, Accuracy: 93.36
Train Epoch: 31 [20480/50000 (45%)]	Loss: 0.208352, Accuracy: 91.02
Train Epoch: 31 [23040/50000 (51%)]	Loss: 0.189197, Accuracy: 92.58
Train Epoch: 31 [25600/50000 (57%)]	Loss: 0.176023, Accuracy: 93.75
Train Epoch: 31 [28160/50000 (62%)]	Loss: 0.160593, Accuracy: 93.36
Train Epoch: 31 [30720/50000 (68%)]	Loss: 0.143138, Accuracy: 95.70
Train Epoch: 31 [33280/50000 (74%)]	Loss: 0.110989, Accuracy: 95.70
Train Epoch: 31 [35840/50000 (80%)]	Loss: 0.233871, Accuracy: 93.36
Train Epoch: 31 [38400/50000 (85%)]	Loss: 0.184241, Accuracy: 94.14
Train Epoch: 31 [40960/50000 (91%)]	Loss: 0.207926

Train Epoch: 37 [15360/50000 (34%)]	Loss: 0.187046, Accuracy: 92.97
Train Epoch: 37 [17920/50000 (40%)]	Loss: 0.149007, Accuracy: 94.92
Train Epoch: 37 [20480/50000 (45%)]	Loss: 0.151884, Accuracy: 94.14
Train Epoch: 37 [23040/50000 (51%)]	Loss: 0.136905, Accuracy: 94.53
Train Epoch: 37 [25600/50000 (57%)]	Loss: 0.196134, Accuracy: 94.14
Train Epoch: 37 [28160/50000 (62%)]	Loss: 0.091243, Accuracy: 96.09
Train Epoch: 37 [30720/50000 (68%)]	Loss: 0.108841, Accuracy: 95.70
Train Epoch: 37 [33280/50000 (74%)]	Loss: 0.237529, Accuracy: 92.58
Train Epoch: 37 [35840/50000 (80%)]	Loss: 0.137737, Accuracy: 95.70
Train Epoch: 37 [38400/50000 (85%)]	Loss: 0.217651, Accuracy: 92.97
Train Epoch: 37 [40960/50000 (91%)]	Loss: 0.090384, Accuracy: 96.48
Train Epoch: 37 [43520/50000 (97%)]	Loss: 0.205005, Accuracy: 93.36

Validation set: Average loss: 0.4020, Accuracy: 4416/5000 (88.00%)

the time of this epoch:[158.25844287872314 s]
Train Epoch: 38 [0/50000 (0%)]	Loss: 0.122125, Accuracy: 95.31
Train 

Train Epoch: 43 [25600/50000 (57%)]	Loss: 0.077322, Accuracy: 98.44
Train Epoch: 43 [28160/50000 (62%)]	Loss: 0.096266, Accuracy: 97.27
Train Epoch: 43 [30720/50000 (68%)]	Loss: 0.084976, Accuracy: 96.88
Train Epoch: 43 [33280/50000 (74%)]	Loss: 0.099587, Accuracy: 97.27
Train Epoch: 43 [35840/50000 (80%)]	Loss: 0.101381, Accuracy: 94.14
Train Epoch: 43 [38400/50000 (85%)]	Loss: 0.089588, Accuracy: 97.27
Train Epoch: 43 [40960/50000 (91%)]	Loss: 0.130441, Accuracy: 95.70
Train Epoch: 43 [43520/50000 (97%)]	Loss: 0.143852, Accuracy: 94.53

Validation set: Average loss: 0.4364, Accuracy: 4406/5000 (88.00%)

the time of this epoch:[158.3684425354004 s]
Train Epoch: 44 [0/50000 (0%)]	Loss: 0.108352, Accuracy: 96.48
Train Epoch: 44 [2560/50000 (6%)]	Loss: 0.060919, Accuracy: 98.44
Train Epoch: 44 [5120/50000 (11%)]	Loss: 0.117291, Accuracy: 95.70
Train Epoch: 44 [7680/50000 (17%)]	Loss: 0.113265, Accuracy: 95.70
Train Epoch: 44 [10240/50000 (23%)]	Loss: 0.072912, Accuracy: 97.27
Train Epoch

Train Epoch: 49 [35840/50000 (80%)]	Loss: 0.125853, Accuracy: 95.31
Train Epoch: 49 [38400/50000 (85%)]	Loss: 0.084665, Accuracy: 96.88
Train Epoch: 49 [40960/50000 (91%)]	Loss: 0.085715, Accuracy: 97.27
Train Epoch: 49 [43520/50000 (97%)]	Loss: 0.114330, Accuracy: 96.09

Validation set: Average loss: 0.4421, Accuracy: 4422/5000 (88.00%)

the time of this epoch:[158.38690209388733 s]
Train Epoch: 50 [0/50000 (0%)]	Loss: 0.075963, Accuracy: 97.27
Train Epoch: 50 [2560/50000 (6%)]	Loss: 0.058694, Accuracy: 97.66
Train Epoch: 50 [5120/50000 (11%)]	Loss: 0.116228, Accuracy: 96.48
Train Epoch: 50 [7680/50000 (17%)]	Loss: 0.087024, Accuracy: 96.48
Train Epoch: 50 [10240/50000 (23%)]	Loss: 0.054119, Accuracy: 98.05
Train Epoch: 50 [12800/50000 (28%)]	Loss: 0.098669, Accuracy: 96.48
Train Epoch: 50 [15360/50000 (34%)]	Loss: 0.103654, Accuracy: 96.88
Train Epoch: 50 [17920/50000 (40%)]	Loss: 0.100731, Accuracy: 95.70
Train Epoch: 50 [20480/50000 (45%)]	Loss: 0.059902, Accuracy: 97.27
Train Epoc

# Step 3: Test

In [10]:
test(epoch)


Test set: Average loss: 0.4947, Accuracy: 8849/10000 (88.49%)



## 第一次 scale 位于[0,1]

![](http://op4a94iq8.bkt.clouddn.com/18-7-14/70206949.jpg)